In [14]:
import sys, os
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from collections import OrderedDict
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../')

from src.objs import ProfileHandler, DatasetProfile
from src.utils import paths_manager as pathsman

In [15]:
profile_path = None

In [16]:
if profile_path:
        profile = DatasetProfile.extract(profile_path=profile_path)
else:
        available_profiles = sorted(list(os.listdir(pathsman.PROFILES_DIR)))
        recent_profile = available_profiles[-1]
        profile = DatasetProfile.extract(pathsman.PROFILES_DIR / recent_profile)

In [17]:
def get_mae(target, prediction):
    return mean_absolute_error(target, prediction)

def get_eac(target, prediction):
    num = np.sum(np.abs(prediction-target))
    den = (np.sum(target))
    eac = 1 - (num/den)/2
    return np.where(eac<0, 0, eac)

def get_relative_error(target, prediction):
    return np.mean(np.nan_to_num(np.abs(target - prediction) / np.maximum(target, prediction)))


def get_nde(target, prediction):
    return np.sum((target - prediction) ** 2) / np.sum((target ** 2))

def compute_regress_metrics(target, prediction):
    eac = get_eac(target, prediction)
    mae = get_mae(target, prediction)
    nade = get_nde(target, prediction)
    
   
    metrics = OrderedDict([('EAC', eac),
                        ('MAE', mae),
                        ('NDE', nade)])
    
    metrics_dict = {}
    metrics_dict['EAC'] = metrics["EAC"]
    metrics_dict['MAE'] = metrics["MAE"]
    metrics_dict['NDE'] = metrics["NDE"]
    
    return metrics_dict

appliances=['fridge', 'washer dryer','kettle', 'dish washer', 'microwave'] 

In [18]:
result_unet = np.load("../results/results.npy", allow_pickle=True).item()

In [19]:
pred_power=result_unet['pred_power']
pred_state=result_unet['pred_state']
power=result_unet['power']
state=result_unet['state']

In [20]:
def smape_score(target, prediction):
    denom = np.abs(prediction) + np.abs(target) 
    score =(2 * np.abs(prediction - target).mean(0))/(denom.mean(0) + 1e-8)
    return score  

In [21]:
y_t = power 
y_p =  pred_power[:,2,:]
def get_CEP(y_t, y_p):
    c = np.sum((y_t == y_p).astype(int)*y_p, 0)
    ce = np.sum((y_t > y_p).astype(int)*y_t, 0)
    co = np.sum((y_t < y_p).astype(int)*y_p, 0)
    gt = np.sum(y_t, 0)
    CEP = (c + ce + co)[gt!=0]/(gt[gt!=0])
    CEP[gt==0]=1.0
    return CEP
gt = np.sum(y_t, 0)
O = np.sum( (y_p - y_t), 0)
U = np.sum( (y_t - y_p), 0)

OE = O[gt!=0]/(gt[gt!=0])
OE

array([1.13532768e-01, 2.04142696e+02, 3.16977960e+02, 1.46878024e-01,
       1.38148647e+03])

In [22]:
smape_score(power, pred_power[:,2,:])

array([2.87147654e+08, 6.92929575e+06, 3.39043011e+06, 1.92090016e+06,
       8.41300353e+06])

In [23]:
appliance_data = profile.metadata

In [24]:
pred_power=result_unet['pred_power']
pred_state=result_unet['pred_state']
power=result_unet['power']
state=result_unet['state']
all_reg_results_mean = []
all_reg_results_std = []
regress = {}
for idx, app in enumerate(list(appliance_data.keys())):
    regress_results = []
    for  q in range(5):
        result = compute_regress_metrics(power[:,idx]*state[:,idx], pred_power[:,q,idx]*pred_state[:,idx])
        result = pd.DataFrame.from_dict(result, orient="index")
        regress_results.append(result)
    regress_results = pd.concat(regress_results, axis=1)
    #regress_results.columns = list(appliance_data.keys())
    #regress[app]=regress_results
    print("APP", app)
    mean = regress_results.mean(1).round(3)
    std  = regress_results.std(1).round(3)
    reg=pd.concat([mean, std], axis=1)
    reg.columns =["mean", 'std']
    all_reg_results_mean.append( mean)
    all_reg_results_std.append(std)
    print(reg)
    print("")
#ebf1 = {"ExbF1":example_f1_score(state, pred_state, axis=0, per_sample=True).tolist()}
#ebf1 =pd.DataFrame.from_dict(ebf1, orient="index")
#ebf1.columns = list(appliance_data.keys())
print("")
#print(ebf1.round(3))


#print(pd.DataFrame.from_dict(compute_metrics(state, pred_state), orient="index"))

APP fridge freezer
      mean    std
EAC  0.370  0.250
MAE  8.606  5.271
NDE  1.997  2.078

APP washer dryer
        mean      std
EAC    0.357    0.254
MAE  576.279  437.458
NDE    1.934    2.061

APP kettle
        mean  std
EAC    0.500  0.0
MAE  225.246  0.0
NDE    1.000  0.0

APP dish washer
        mean      std
EAC    0.549    0.123
MAE  606.468  164.819
NDE    0.823    0.363

APP microwave
       mean  std
EAC   0.500  0.0
MAE  51.688  0.0
NDE   1.000  0.0


